# G2Net Gravitational Wave Detection with FastAI

## Initialize and Load Files

In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path("./")
path.ls()
Path.BASE_PATH = path

In [ ]:
files = get_image_files(path/'traincqt')
len(files)

### Data was pre-processed as CQT

In [ ]:
labels = pd.read_csv(path/'training_labels.csv')
labels['id'] = labels['id'].map(lambda x : f'{path}/traincqt/{x}.png' )
labels.head()

## Helper Functions for FastAI DataBlock

In [ ]:
def getx(df):
    return df['id']

def gety(df):
    return df['target']

In [ ]:
def custom_splitter(train_pct):
    def fn(name_list):
        train_idx, valid_idx = RandomSplitter(valid_pct=0.3, seed=42)(name_list)
        np.random.shuffle(train_idx)
        train_len = int(len(train_idx) * train_pct)
        np.random.shuffle(valid_idx)
        valid_len = int(len(valid_idx) * train_pct)
        return train_idx[0:train_len], valid_idx[0:valid_len]
    return fn

In [ ]:
def build_data(df, bs=32, train_pct=.05):
    grav = DataBlock(blocks = (ImageBlock,CategoryBlock),
                 get_x  = getx,
                 get_y  = gety,
                 splitter = custom_splitter(train_pct),
                 #item_tfms = Resize(460),
                 batch_tfms = [*aug_transforms(do_flip=False, max_rotate=0),
                               Normalize.from_stats(*imagenet_stats)]) #, 
                #splitter=RandomSplitter(valid_pct=0.3, seed=42))
    grav.summary(df)
    return grav.dataloaders(df, bs=bs)

## Build DataBlock and show batch

In [ ]:
dls = build_data(labels, train_pct=1)
dls.show_batch()

## Make Learner, Find Good Learning Rate

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=RocAucBinary(), cbs=[SaveModelCallback(with_opt=True)])
learn.lr_find()

## Fine Tune

In [ ]:
learn.fine_tune(4, base_lr=6e-5)

## Save Model and Create Submission Output

In [ ]:
save_model(path/'models/modelwithaug', learn, learn.opt)

In [ ]:
sample = pd.read_csv(path/'sample_submission.csv')
sampledf = sample.copy()
sampledf['id'] = sampledf['id'].map(lambda x: f'{path}/testcqt/{x}.png')
sampledf

In [ ]:
test_dl = dls.test_dl(sampledf)
preds, _ = learn.get_preds(dl=test_dl)

In [ ]:
sample.target = np.array(preds[:,1])
sample.to_csv('submission8.csv', index=False)
sample.head()